In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:

#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os

from sklearn import model_selection
from sklearn.metrics import f1_score

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
 
import albumentations as A
from albumentations.pytorch import ToTensorV2

import pydicom
import cv2
from tqdm import tqdm

In [3]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [4]:
class CFG:
    model_name= 'convnext_tiny'
    target_size= 2
    lr= 1e-4
    weight_decay= 1e-6
    
    T_0=10 
    min_lr=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000

In [5]:
train_csv= pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
train_csv.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True


In [6]:
train_csv['cancer'].value_counts()

0    53548
1     1158
Name: cancer, dtype: int64

In [7]:
path= '/kaggle/input/rsna-breast-cancer-256-pngs'
for i in range(len(train_csv)):
    s=str(train_csv.loc[i,'patient_id'])+'_'+str(train_csv.loc[i,'image_id'])+'.png'
    img_path=os.path.join(path,s)
    train_csv.loc[i,'img_path']= img_path

In [8]:
canc= train_csv[train_csv.cancer==1]
noncanc= train_csv[train_csv.cancer==0]
noncanc= noncanc.reset_index(drop=True)
canc= canc.reset_index(drop= True)
# from sklearn.utils import resample
# canc_upsample = resample(canc,
#              replace=True,
#              n_samples=len(noncanc),
#              random_state=42)

new_df= pd.DataFrame()
#new_df= pd.concat([canc_upsample, noncanc])
#new_df= pd.concat([canc, noncanc[:1500]])
new_df= train_csv
new_df= new_df.sample(frac=1)
new_df= new_df.reset_index(drop=True)

In [9]:
group_kfold = model_selection.GroupShuffleSplit(n_splits=1,test_size= 0.1)
for i, (train_index, test_index) in enumerate(group_kfold.split(new_df, new_df.cancer, new_df.patient_id)):
    new_df.loc[train_index, 'group'] = i
    new_df.loc[test_index , 'group'] = i+1

new_df['group'] = new_df['group'].astype(int)
new_df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,img_path,group
0,2,57789,255254701,L,MLO,49.0,0,0,0,NaN,0,NaN,48,False,/kaggle/input/rsna-breast-cancer-256-pngs/5778...,0
1,1,15361,1167501842,L,MLO,46.0,0,0,0,1.0,1,C,49,False,/kaggle/input/rsna-breast-cancer-256-pngs/1536...,0
2,2,22922,584307441,R,MLO,66.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-256-pngs/2292...,0
3,1,38494,713816140,L,CC,73.0,0,0,0,2.0,0,B,93,False,/kaggle/input/rsna-breast-cancer-256-pngs/3849...,0
4,1,57790,1893007795,R,CC,56.0,0,0,0,2.0,0,B,49,False,/kaggle/input/rsna-breast-cancer-256-pngs/5779...,0


In [10]:
train_df= new_df[new_df.group==0]
test_df= new_df[new_df.group==1]

train_df= train_df.reset_index(drop=True)
test_df= test_df.reset_index(drop=True)

In [11]:
skf = model_selection.StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(skf.split(train_df, train_df.cancer)):
    train_df.loc[test_index,'fold']= i
    
train_df['fold']= train_df['fold'].astype(int)
train_df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,img_path,group,fold
0,2,57789,255254701,L,MLO,49.0,0,0,0,NaN,0,NaN,48,False,/kaggle/input/rsna-breast-cancer-256-pngs/5778...,0,0
1,1,15361,1167501842,L,MLO,46.0,0,0,0,1.0,1,C,49,False,/kaggle/input/rsna-breast-cancer-256-pngs/1536...,0,0
2,2,22922,584307441,R,MLO,66.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-256-pngs/2292...,0,0
3,1,38494,713816140,L,CC,73.0,0,0,0,2.0,0,B,93,False,/kaggle/input/rsna-breast-cancer-256-pngs/3849...,0,0
4,1,57790,1893007795,R,CC,56.0,0,0,0,2.0,0,B,49,False,/kaggle/input/rsna-breast-cancer-256-pngs/5779...,0,0


In [12]:
train_df.groupby(['fold','group']).cancer.value_counts()

fold  group  cancer
0     0      0         9639
             1          205
1     0      0         9639
             1          205
2     0      0         9638
             1          205
3     0      0         9638
             1          205
4     0      0         9638
             1          205
Name: cancer, dtype: int64

In [13]:
def get_df(df, i):
    pos= df[df.cancer==1]
    neg= df[df.cancer==0]
    pos= pos.reset_index(drop=True)
    neg= neg.reset_index(drop=True)
    
    start= (i-1)*len(pos)
    end= i*len(pos)
    neg_new= neg[start: end]
    n= pd.DataFrame()
    n= pd.concat([neg_new,pos])
    n= n.sample(frac=1)
    n.reset_index(drop= True)
    
    return n

In [14]:
class TrainDataset(Dataset):
    def __init__(self, df, transform=None ):
        self.transform = transform
        self.img_path= df['img_path'].values 
        self.labels=df['cancer'].values
    
    def __len__(self):
        return len(self.img_path)
    
    def __getitem__(self, item):
        image = cv2.imread(self.img_path[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            aug = self.transform(image=image)
            image=aug['image']
        
        classCategory = torch.tensor(self.labels[item]).long()
        return image, classCategory

In [15]:
# train_dataset= TrainDataset(X_train,transform=None)
# img,clas= train_dataset[0]
# img.shape

In [16]:
def get_transforms(data):
    
    if data == 'train':
        return A.Compose([
            A.Resize(224, 224),
            A.RandomResizedCrop(224, 224),
            A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    if data == 'valid':
        return A.Compose([
            A.Resize(224, 224),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
            ToTensorV2()
                ])

In [17]:
# train_dataset= TrainDataset(X_train, transform= get_transforms('train'))
# img,clas= train_dataset[0]
# img.shape

In [18]:
class CustomNet(nn.Module):
    def __init__(self, model_name=CFG.model_name, pretrained=False):
        super().__init__()
        self.model = timm.create_model(CFG.model_name, pretrained=pretrained)
        #print(self.model)
        in_features = self.model.head.fc.in_features
        self.model.head.fc= nn.Linear(in_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x

In [19]:
def get_score(labels, preds, beta=1):
    preds = preds.clip(0, 1)
    y_true_count = labels.sum()
    ctp = preds[labels==1].sum()
    cfp = preds[labels==0].sum()
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0.0

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def train_one_epoch(train_loader, model, criterion, optimizer, epoch, scheduler, device='cpu'):
    losses = AverageMeter()
    preds=[]
    truths=[]
    global_step = 0
    
    model.train()
    model=model.to(device)
    for step, (imgs, labels) in tqdm(enumerate(train_loader),total= len(train_loader)):
        imgs= imgs.to(device)
        labels= labels.to(device)
        batch_size = labels.size(0)
        y_preds= model(imgs)
        preds.append(y_preds.softmax(1).to('cpu').detach().numpy())   
        truths.append(labels.to('cpu').numpy())
        
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps  
        
        loss.backward()    
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        
    preds= np.concatenate(preds)
    predictions = np.argmax(preds,axis=1)
    truths= np.concatenate(truths)
    #print(preds,truths)
    score= get_score(truths ,predictions)
    print(f"Epoch: {epoch} Train_loss: {losses.avg} Train_F1: {score}")
    return score

def val_one_epoch(val_loader, model, criterion, epoch, device='cpu'):
    losses = AverageMeter()
    preds=[]
    truths=[]
    
    model.eval()
    model=model.to(device)
    for step, (imgs, labels) in tqdm(enumerate(val_loader),total= len(val_loader)):
        imgs= imgs.to(device)
        labels= labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds= model(imgs)
            preds.append(y_preds.softmax(1).to('cpu').numpy())  
            truths.append(labels.to('cpu').numpy())

            loss = criterion(y_preds, labels)
            losses.update(loss.item(), batch_size)
            if CFG.gradient_accumulation_steps > 1:
                loss = loss / CFG.gradient_accumulation_steps
    
    predictions = np.argmax(np.concatenate(preds),axis=1)
    truths= np.concatenate(truths)
    score= get_score(truths ,predictions)
    print(f"Epoch: {epoch} Val_loss: {losses.avg} Val_F1: {score}")
    return score

In [20]:
# device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model= CustomNet(pretrained= True)
# optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
# scheduler = ExponentialLR(optimizer, gamma=0.9)
# criterion =nn.CrossEntropyLoss().to(device)

# train_dataset= TrainDataset(X_train, transform= get_transforms('train'))
# train_loader= DataLoader(train_dataset, batch_size=64,
#                          shuffle=True,
#                          num_workers=1)
# pred= val_one_epoch(train_loader,model, criterion,0)

In [21]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss()(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [22]:
def train(df,epochs,fold):
    print(f"Fold: {fold}")
    device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model= CustomNet(pretrained= True)
    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    #scheduler = ExponentialLR(optimizer, gamma=0.8)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
    criterion =FocalLoss().to(device)
    best_score=-1
    
    X_train= df.loc[df['fold']!=fold]
    X_test= df.loc[df['fold']==fold]
    X_train= X_train.reset_index(drop=True)
    X_test= X_test.reset_index(drop=True)

    train_dataset= TrainDataset(X_train, transform= get_transforms('train'))
    train_loader= DataLoader(train_dataset, batch_size=64,
                                 shuffle=True,
                                 num_workers=2)
    val_dataset= TrainDataset(X_test, transform= get_transforms('valid'))
    val_loader= DataLoader(val_dataset, batch_size=64,
                                 shuffle=True,
                                 num_workers=2)
    for epoch in range(epochs):
        train_score=train_one_epoch(train_loader, model, criterion, optimizer, epoch, scheduler, device)
        val_score= val_one_epoch(val_loader,model, criterion,epoch, device)
        scheduler.step()
        if val_score>best_score:
            best_score= val_score
            print("Model improved :D\n")
            model= model.to('cpu')
            torch.save(model.state_dict(),f'model{fold}.pth')

In [23]:
# u,v = model_selection.train_test_split(train_csv,stratify=train_csv.cancer, test_size=0.9, random_state=42)
# neg= v[v['cancer']==0]
# pos= v[v['cancer']==1]

# t= pd.DataFrame()
# t= pd.concat([pos, neg[:1500]])
# t= t.sample(frac=1)
# t= t.reset_index(drop= True)
# len(t)

In [24]:
#u,v = model_selection.train_test_split(train_df,stratify=train_df.fold, test_size=0.001, random_state=42)
for i in range(5):
    train(train_df, 10, i)

Fold: 0


Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny_1k_224_ema.pth
100%|██████████| 616/616 [13:30<00:00,  1.32s/it]

Epoch: 0 Train_loss: 0.002613615002827855 Train_F1: 0.004629629629629629



100%|██████████| 154/154 [00:54<00:00,  2.85it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in long_scalars
  import sys


Epoch: 0 Val_loss: 0.0018658511953111262 Val_F1: 0.0
Model improved :D



...

Epoch: 8 Val_loss: 0.0018439076799708091 Val_F1: 0.0



100%|██████████| 616/616 [13:32<00:00,  1.32s/it]

Epoch: 9 Train_loss: 0.0018363828881071459 Train_F1: 0.0



100%|██████████| 154/154 [00:34<00:00,  4.49it/s]


Epoch: 9 Val_loss: 0.0017847101954463387 Val_F1: 0.0


In [25]:
def test(test_dataset, model, device='cpu'):
    preds=[]
    
    model.eval()
    model=model.to(device)
    for i in tqdm(range(len(test_dataset))):
        img,label= test_dataset[i]
        img= img.unsqueeze(0)
        img= img.to(device)
        label= label.to(device)
        with torch.no_grad():
            y_pred= model(img)
            preds.append(y_pred.softmax(1).to('cpu').numpy())   

    predictions = np.argmax(np.concatenate(preds),axis=1)
    return predictions

In [26]:
test_dataset= TrainDataset(test_df, transform= get_transforms('valid'))
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tot_pred=[]
tot_truth=[]

for fold in range(5):
    model= CustomNet(pretrained= False)
    model.load_state_dict(torch.load(f'/kaggle/working/model{fold}.pth'))
    preds=  test(test_dataset,model,device)
    tot_pred.append(preds)
    
tot_pred= np.vstack(tot_pred)
tot_pred= np.max(tot_pred,axis=0)
tot_truth= test_df.cancer.values
    
score= get_score(tot_truth, tot_pred)
print(f"Holdout f1_score: {score}")

100%|██████████| 5489/5489 [00:56<00:00, 98.01it/s]

Holdout f1_score: 0.0



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in long_scalars
  import sys
